<a href="https://colab.research.google.com/github/ScarletBird/pizza_shop_chatbot/blob/main/Pizza_shop_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this project, the NLTK chat class was used to create a simple chatbot, which will interact with the user, allowing them to: view the menu, remove or add items to their order, and finish their order. The bot is made in Portuguese.

In [ ]:
# Basic libraries import
import nltk
from nltk.chat.util import Chat, reflections 

# Class chat's modification requirements
import re
import random

import csv

Before starting the bot's construction, an adaptation is required so the chatbot will be able to call functions when called (add or remove items). After a quick search, the following method was found:

In [ ]:
class MyChat(Chat):

    def __init__(self, pairs, reflections={}):

        # Now that we have 4 items, being the third a function (adding z) and the fourth arguments for the function (adding args)
        self._pairs = [(re.compile(x, re.IGNORECASE), y, z, args) for (x, y, z, args) in pairs]
        self._reflections = reflections
        self._regex = self._compile_reflections()

    def respond(self, str):

        # Here there is the 'callback', which will call the function
        for (pattern, response, callback, args) in self._pairs:
            match = pattern.match(str)

            if match:

                resp = random.choice(response)
                resp = self._wildcards(resp, match)

                if resp[-2:] == '?.':
                    resp = resp[:-2] + '.'
                if resp[-2:] == '??':
                    resp = resp[:-2] + '?'

                # if a 'callback' does exist, it will be executed here  
                if callback:
                  if args:
                    callback(args)
                  else: # if the function has no arguments, this method is called instead
                    callback()

                return resp

First the data that will be used for universal and basic commands (don't need functions to word) will be imported by csv. The menu, in case the client requests it, is also in a csv.

In [ ]:
# Pairs are added as the code goes
pares = []

# The universal commands file is read to associate in the pairs
with open('Pizza_Universal.csv', newline='') as File:  
  reader = csv.reader(File, delimiter=';')
  for linha in reader:
    add = [
      r"{}".format(linha[0]), # Regular expressions (user part)
      [], # Answer
      None, # For universal answers, no functions is executed
      None # And no argument is requiered either
    ]
    # Add each answer, divided by bar, to the tuple "add[1]"
    for x in linha[1].split('|'): add[1].append(x)
    # Add "add" in "pairs"
    pares.append(add)
# Debug
#print(pares)

Adding the menu so the client can order through the pairs, besides allowing the client to request the menu by chat.

In [ ]:
menu = []

# CSV reading as above
with open('Pizza_Menu.csv', newline='') as File:  
  reader = csv.reader(File, delimiter=';')
  for linha in reader:
    menu.append([linha[0],linha[1],linha[2]]) # Here are added: the pizza, the ingredients, and the price

def mostrarMenu():
  # The menu has a different format to make reading friendlier to the user, by fixing spaces
  print('{0: <23} | {1: <70} | {2}'.format('Pizza', 'Ingredientes da pizza', 'Preço'))
  print('_'*108)
  for p, i, pr in menu:
    print('Pizza de {0: <14} | {1: <70} | R$ {2:.2f}'.format(p,i,float(pr)))

# Pair for the client to request the menu (the bot's answer will always come after the function's execution)
pares.append(
    [
      r'(.*)\bmenu\?*',
      ['Claro, segue o menu. Posso te ajudar em algo mais?'],
      mostrarMenu,
      None
    ]
)

# Quantity, both spelled and written
quantidade = {
    "um": 1,
    "uma": 1,
    "dois": 2,
    "duas": 2,
    "três": 3,
    "tres": 3,
    "quatro": 4,
    "cinco": 5,
    "seis": 6,
    "sete": 7,
    "oito": 8,
    "nove": 9
}
quantidade_n = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
#print(quantidade)
#print(menu)

Now the functions that will be used by the chatbot will be added. With these functions, the client will be able to add, remove, and show items.

In [ ]:
# Client's order starts empty
pedido = []

# Basic function to add items by desired quantity
def adicionarItem(args):
  item, preço, quantidade = args
  pedido.append([item, quantidade, preço*quantidade])

# Function to remove an item by a quantity
def retirarItem(args):
  item, preço, quantidade = args
  for i in range(len(pedido)):
    if pedido[i][0] == item:
      if pedido[i][1] <= quantidade:
        del(pedido[i])
      else:
        pedido[i][1] = pedido[i][1] - quantidade
        pedido[i][2] = pedido[i][2] - preço*quantidade
      break

# Function that deletes all the order's items
def limparPedido():
  pedido.clear()

# Show the order in a friendly format
def mostrarPedido():
  pTotal = 0
  print('_' * 20)
  if len(pedido) > 0:
    print('O seu pedido está com:')
    for i, q, p in pedido:
      print('{0: <14} | {1} por R$ {2:.2f}'.format(i.capitalize(), q, float(p)))
      pTotal = pTotal + p
    print('_' * 20)  
    print('Total do pedido: R$ {0:.2f}'.format(pTotal))
  else: print("Seu pedido está vazio!")


Now the pairs will be added, with the menu and the already defined functions.

In [ ]:
# For each pizza, a pair with its quantity will be added (1 to 9)
for pizza, i, p in menu:
  for key, value in quantidade.items():
    # A pair to add an item
    add = [
        r'{0} {1}s?'.format(key, pizza),
        ['{0} pizza(s) de {1} adicionado ao pedido'.format(key.capitalize(), pizza)],
        adicionarItem,
        [pizza, float(p), value]
    ]

    # And a pair to remove
    add_retirar = [
        r'retire {0} {1}s?'.format(key, pizza),
        ['{0} pizza(s) de {1} removida do pedido'.format(key.capitalize(), pizza)],
        retirarItem,
        [pizza, float(p), value]
    ]
    pares.append(add)
    pares.append(add_retirar)

  # Same logic as above, but using spelled numbers
  for n in quantidade_n:
    add = [
        r'{0} {1}s?'.format(n, pizza),
        ['{0} pizza(s) de {1} adicionado ao pedido'.format(n, pizza)],
        adicionarItem,
        [pizza, float(p), int(n)]
    ]
    add_retirar = [
        r'retire {0} {1}s?'.format(n, pizza),
        ['{0} pizza(s) de {1} removida do pedido'.format(n, pizza)],
        retirarItem,
        [pizza, float(p), int(n)]
    ]
    pares.append(add)
    pares.append(add_retirar)

# Command so the client can see his order
pares.append([
      r'(.*)\bmeu pedido\?*',
      ['Posso te ajudar com algo mais?'],
      mostrarPedido,
      None
])

# Function to define if the client will finish his order, cleaning it. In order to make it functional, a message should be sent to another system
def concluirPedido():
  mostrarPedido() # Shows the order before finishing it

  # Positive patterns will be accepted by the system to complete the order
  positivo = re.compile('(s(im)?)|(claro)|(pode?)|(por favor)|(pfv)|(com certeza)|(com ctz)|(fech(ar|e))|(conclu(ir|a))|(encerr(ar|e))')
  encerrar = input('Posso encerrar o pedido? ')
  
  # If the answer is positive, the order will be completed, or else, nothing happens
  if positivo.search(encerrar.lower()):
    print('Pedido concluído! Prazo de entrega: 30 min. Agredecemos pelo seu pedido!')
    limparPedido()

# To complete the order, the bot will accept the following words
pares.append([
      r'(.*)\b((finalizar)|(encerrar)|(concluir)|(isso [eé] tudo)|(s[oó] isso))\?*',
      ['Posso te ajudar em algo mais?'],
      concluirPedido,
      None
])

#print(pares[4])

After all parts are concluded, the chat will start after the command bellow:

In [ ]:
chat = MyChat(pares,reflections) # MyChat is used instead of Chat, since there are more functions to it
chat.converse()

>Olá
Oi! Boas vindas à pizzaria Pizza!
>Quero ver o menu
Pizza                   | Ingredientes da pizza                                                  | Preço
____________________________________________________________________________________________________________
Pizza de mussarela      | Mussarela, molho de tomate e orégano                                   | R$ 10.50
Pizza de calabresa      | Calabresa, molho de tomate, cebola e orégano                           | R$ 10.50
Pizza de marguerita     | Mussarela, tomate, molho de tomate, orégano e manjericão               | R$ 10.50
Pizza de portuguesa     | Mussarela, presunto, ovos, ervilha, cebola, molho de tomate e orégano  | R$ 13.50
Pizza de atum           | Atum, cebola e molho de tomate                                         | R$ 10.50
Pizza de napolitana     | Mussarela, parmesão, tomate e molho de tomate                          | R$ 11.50
Pizza de palmito        | Mussarela, palmito e molho de tomate                   